**Prepare the Dataset**

In [ ]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

In [ ]:
def load_utkface_data(data_path):
    images = []
    ages = []
    for filename in os.listdir(data_path):
        if filename.endswith(".jpg"):
            age = int(filename.split("_")[0])
            img = cv2.imread(os.path.join(data_path, filename))
            img = cv2.resize(img, (200, 200))
            images.append(img)
            ages.append(age)
    return np.array(images), np.array(ages)

In [ ]:
data_path = 'UTKFace'
images, ages = load_utkface_data(data_path)

Streaming output truncated to the last 5000 lines.
/content/UTKFace/12_1_3_20170104223448631.jpg.chip.jpg
/content/UTKFace/1_0_2_20161219202542596.jpg.chip.jpg
/content/UTKFace/26_1_0_20170117170217033.jpg.chip.jpg
/content/UTKFace/40_0_4_20170117135456629.jpg.chip.jpg
/content/UTKFace/22_0_1_20170114033114683.jpg.chip.jpg
/content/UTKFace/26_0_4_20170117195459004.jpg.chip.jpg
/content/UTKFace/24_1_3_20170119171107656.jpg.chip.jpg
/content/UTKFace/37_0_0_20170113210126299.jpg.chip.jpg
/content/UTKFace/1_1_2_20161219194854052.jpg.chip.jpg
/content/UTKFace/32_1_2_20170116190147549.jpg.chip.jpg
/content/UTKFace/26_0_1_20170117195937173.jpg.chip.jpg
/content/UTKFace/72_1_0_20170110125258812.jpg.chip.jpg
/content/UTKFace/1_0_2_20161219141023272.jpg.chip.jpg
/content/UTKFace/26_0_3_20170119183051629.jpg.chip.jpg
/content/UTKFace/26_0_3_20170119181430244.jpg.chip.jpg
/content/UTKFace/27_0_1_20170117010723234.jpg.chip.jpg
/content/UTKFace/50_0_1_20170111194904758.jpg.chip.jpg
/content/UTKFace/

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)

**Build and Train the CNN Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  
])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    'age_gender_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=1,
    restore_best_weights=True
)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

**Implement Grad-CAM**

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
def get_img_array(img_path, size):
    img = cv2.imread(img_path)
    img = cv2.resize(img, size)
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    cv2.imwrite(cam_path, superimposed_img)
    return superimposed_img

In [ ]:
img_path = 'path_to_sample_image.jpg'
img_array = get_img_array(img_path, size=(200, 200))
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name="conv2d_2")
superimposed_img = display_gradcam(img_path, heatmap)
plt.imshow(superimposed_img)
plt.show()